# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 30 2022 10:35AM,240831,19,N17670,"NAPP Technologies, LLC",Released
1,Jun 30 2022 10:10AM,240830,20,ALUR870811059,Alumier Labs Inc.,Released
2,Jun 30 2022 10:03AM,240829,20,ALUR640717918,Alumier Labs Inc.,Released
3,Jun 30 2022 9:59AM,240828,19,ACG-2102490308,ACG North America LLC,Released
4,Jun 30 2022 9:39AM,240823,10,0085814597,ISDIN Corporation,Released
5,Jun 30 2022 9:39AM,240823,10,0085814577,ISDIN Corporation,Released
6,Jun 30 2022 9:39AM,240823,10,0085814746,ISDIN Corporation,Released
7,Jun 30 2022 9:39AM,240823,10,0085814758,ISDIN Corporation,Released
8,Jun 30 2022 9:39AM,240823,10,0085814532,ISDIN Corporation,Released
9,Jun 30 2022 9:39AM,240823,10,0085814531,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,240826,Released,18
33,240828,Released,1
34,240829,Released,1
35,240830,Released,1
36,240831,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240826,NaN,18.0
240828,NaN,1.0
240829,NaN,1.0
240830,NaN,1.0
240831,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240750,25.0,1.0
240766,0.0,1.0
240768,0.0,1.0
240769,3.0,1.0
240774,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240750,25,1
240766,0,1
240768,0,1
240769,3,1
240774,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240750,25,1
1,240766,0,1
2,240768,0,1
3,240769,3,1
4,240774,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240750,25,1
1,240766,,1
2,240768,,1
3,240769,3,1
4,240774,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 30 2022 10:35AM,240831,19,"NAPP Technologies, LLC"
1,Jun 30 2022 10:10AM,240830,20,Alumier Labs Inc.
2,Jun 30 2022 10:03AM,240829,20,Alumier Labs Inc.
3,Jun 30 2022 9:59AM,240828,19,ACG North America LLC
4,Jun 30 2022 9:39AM,240823,10,ISDIN Corporation
54,Jun 30 2022 9:37AM,240826,20,Alumier Labs Inc.
72,Jun 30 2022 9:30AM,240807,10,ISDIN Corporation
123,Jun 30 2022 9:27AM,240821,10,Emerginnova
128,Jun 30 2022 9:25AM,240819,10,Bio-PRF
144,Jun 30 2022 9:24AM,240820,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 30 2022 10:35AM,240831,19,"NAPP Technologies, LLC",,1
1,Jun 30 2022 10:10AM,240830,20,Alumier Labs Inc.,,1
2,Jun 30 2022 10:03AM,240829,20,Alumier Labs Inc.,,1
3,Jun 30 2022 9:59AM,240828,19,ACG North America LLC,,1
4,Jun 30 2022 9:39AM,240823,10,ISDIN Corporation,,50
5,Jun 30 2022 9:37AM,240826,20,Alumier Labs Inc.,,18
6,Jun 30 2022 9:30AM,240807,10,ISDIN Corporation,,51
7,Jun 30 2022 9:27AM,240821,10,Emerginnova,,5
8,Jun 30 2022 9:25AM,240819,10,Bio-PRF,,16
9,Jun 30 2022 9:24AM,240820,20,"Exact-Rx, Inc.",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 30 2022 10:35AM,240831,19,"NAPP Technologies, LLC",1,
1,Jun 30 2022 10:10AM,240830,20,Alumier Labs Inc.,1,
2,Jun 30 2022 10:03AM,240829,20,Alumier Labs Inc.,1,
3,Jun 30 2022 9:59AM,240828,19,ACG North America LLC,1,
4,Jun 30 2022 9:39AM,240823,10,ISDIN Corporation,50,
5,Jun 30 2022 9:37AM,240826,20,Alumier Labs Inc.,18,
6,Jun 30 2022 9:30AM,240807,10,ISDIN Corporation,51,
7,Jun 30 2022 9:27AM,240821,10,Emerginnova,5,
8,Jun 30 2022 9:25AM,240819,10,Bio-PRF,16,
9,Jun 30 2022 9:24AM,240820,20,"Exact-Rx, Inc.",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 30 2022 10:35AM,240831,19,"NAPP Technologies, LLC",1,
1,Jun 30 2022 10:10AM,240830,20,Alumier Labs Inc.,1,
2,Jun 30 2022 10:03AM,240829,20,Alumier Labs Inc.,1,
3,Jun 30 2022 9:59AM,240828,19,ACG North America LLC,1,
4,Jun 30 2022 9:39AM,240823,10,ISDIN Corporation,50,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 30 2022 10:35AM,240831,19,"NAPP Technologies, LLC",1.0,NaN
1,Jun 30 2022 10:10AM,240830,20,Alumier Labs Inc.,1.0,NaN
2,Jun 30 2022 10:03AM,240829,20,Alumier Labs Inc.,1.0,NaN
3,Jun 30 2022 9:59AM,240828,19,ACG North America LLC,1.0,NaN
4,Jun 30 2022 9:39AM,240823,10,ISDIN Corporation,50.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 30 2022 10:35AM,240831,19,"NAPP Technologies, LLC",1.0,0.0
1,Jun 30 2022 10:10AM,240830,20,Alumier Labs Inc.,1.0,0.0
2,Jun 30 2022 10:03AM,240829,20,Alumier Labs Inc.,1.0,0.0
3,Jun 30 2022 9:59AM,240828,19,ACG North America LLC,1.0,0.0
4,Jun 30 2022 9:39AM,240823,10,ISDIN Corporation,50.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2167277,142.0,26.0
12,481619,1.0,2.0
16,722424,6.0,3.0
17,240768,1.0,0.0
18,240813,1.0,0.0
19,1204032,38.0,25.0
20,1685581,25.0,50.0
21,1203910,3.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2167277,142.0,26.0
1,12,481619,1.0,2.0
2,16,722424,6.0,3.0
3,17,240768,1.0,0.0
4,18,240813,1.0,0.0
5,19,1204032,38.0,25.0
6,20,1685581,25.0,50.0
7,21,1203910,3.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,142.0,26.0
1,12,1.0,2.0
2,16,6.0,3.0
3,17,1.0,0.0
4,18,1.0,0.0
5,19,38.0,25.0
6,20,25.0,50.0
7,21,3.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,142.0
1,12,Released,1.0
2,16,Released,6.0
3,17,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,17,18,19,20,21
Status,,,,,,,,
Executing,26.0,2.0,3.0,0.0,0.0,25.0,50.0,2.0
Released,142.0,1.0,6.0,1.0,1.0,38.0,25.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,17,18,19,20,21
0,Executing,26.0,2.0,3.0,0.0,0.0,25.0,50.0,2.0
1,Released,142.0,1.0,6.0,1.0,1.0,38.0,25.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,17,18,19,20,21
0,Executing,26.0,2.0,3.0,0.0,0.0,25.0,50.0,2.0
1,Released,142.0,1.0,6.0,1.0,1.0,38.0,25.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()